# Starting notebook

Let us visualize the data and produce low pass filtering

In [ ]:
# Some useful modules for notebooks
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
from tqdne.conf import Config
import h5py
from scipy import signal
import numpy as np
from tqdm import tqdm

from tqdne.dataset import compute_mean_std_features, extract_sample_from_h5file, build_sample

In [ ]:

config = Config()


In [ ]:

datapath = config.datapath
features_keys = config.features_keys
means, stds = compute_mean_std_features(datapath, features_keys)

In [ ]:
# Opent the hdf5 file

sample_id = 0
with h5py.File(datapath, "r") as f:
    for key in f.keys():
        print(f"Key: {key} -- {f[key].shape}")

    time = f["time_vector"][:]
    waveform, features = extract_sample_from_h5file(f, sample_id)
plt.plot(time, waveform.T)
plt.xlabel("Time [s]")
features, stds, means

In [ ]:
fs = config.fs
params_filter = config.params_filter

In [ ]:
# plot the frequency response
b, a = signal.butter(**params_filter, analog=True)
w, h = signal.freqs(b, a)
plt.figure(figsize=(5,3))
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()

In [ ]:

sos = signal.butter(**params_filter, fs=fs, output='sos')
plt.figure(figsize=(8, 8))
for i, channel in enumerate(waveform):
    datafilt = signal.sosfilt(sos, channel)
    plt.subplot(len(waveform), 1, 1+i)
    plt.plot(time, channel , 'b',  alpha=1,)
    plt.plot(time, datafilt,  'r', alpha=1,)
    plt.title(f"Channel {i+1}")
    plt.xlim(np.min(time), np.max(time))
    plt.ylabel("Amplitude")
    plt.xlabel("Time [s]")
    if i == 0:
        plt.legend(["Original","Filtered"])
    # plt.xlim(8, 12)
plt.tight_layout()


In [ ]:

sigma_in = config.sigma_in
sample_id = 0
with h5py.File(datapath, "r") as f:
    time = f["time_vector"][:]
    waveform, features = extract_sample_from_h5file(f, sample_id)
    features, datafilt, waveform, scale = build_sample(waveform, features, means, stds, sos, sigma_in)

n_channels = waveform.shape[0]
plt.figure(figsize=(10, 10))
for i, (filt, orig) in enumerate(zip(datafilt, waveform)):
    plt.subplot(n_channels, 1, 1+i)
    plt.plot(time, filt,  'r', alpha=1, label="normalized low-pass")
    plt.plot(time, orig , 'b',  alpha=0.5, label="normalized target")
    plt.legend()
    # plt.xlim(8, 12
features